In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggerateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = 'train_log/transfer/all_stages/'
logger.set_logger_dir(log_dir)

[1025 09:09:13 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-df6ce0f6-4f02-409d-ad19-18c7cb62956a.json


In [40]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.ckpt"

#### Split train set and test set

In [51]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()

Group numbers:
train: 5014, validation: 0, test: 4726
Image numbers:
train: 22989, validation: 0, test: 21028


#### Gether data to train rnn

In [60]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.annotation_number = 30
dm = DataManager.from_dataset(train_set, config)

Group numbers:
train: 3583, validation: 829, test: 0
Image numbers:
train: 16164, validation: 3651, test: 0


In [68]:
dm.get_imbalance_ratio()

/home/fuxiaofeng/Documents/flyexpress/DL_biomedicine_image/utils/data_manager/__init__.py:142: RuntimeWarning: invalid value encountered in double_scalars
  posi_ratio = np.sum(binary_annot, axis=0) / binary_annot.shape[0]


,train,val,test
ubiquitous,1.611516,1.467262,NaN
faint ubiquitous,4.082270,3.764368,NaN
embryonic midgut,6.011742,6.146552,NaN
trunk mesoderm primordium,6.823144,6.271930,NaN
posterior midgut primordium,8.140306,8.109890,NaN
ventral nerve cord,8.657682,8.314607,NaN
anterior midgut primordium,8.478836,8.528736,NaN
embryonic brain,8.980501,8.639535,NaN
cellular blastoderm,9.538235,9.109756,NaN
embryonic hindgut,10.447284,10.842857,NaN


In [70]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64

train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
model = RNN(config)